Tugas 3 - Implementasi Hasil VSM dengan Algoritma Logistic Regression

Nama : Isnita Widyur Rahmah
NIM : 220411100048
Kelas : IF 7A

Link Project : https://github.com/taawdyy/ppw

In [ ]:
!pip install Sastrawi pandas requests beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import requests
import re

from urllib.request import urlopen
from bs4 import BeautifulSoup

from tqdm import tqdm

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import seaborn as sns
import matplotlib.pyplot as plt

import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
file_path = 'crawl_berita.csv'

df = pd.read_csv(file_path)

df

,Judul,Isi,Tanggal,Kategori
0,Pertemuan Sri Mulyani-Prabowo Tak Banyak Bahas...,Wakil Menteri Keuangan II Thomas Djiwandono me...,"Rabu, 11 Sep 2024 18:10 WIB",Ekonomi
1,Pebisnis Minta Jokowi Cabut Larangan Jual Roko...,Gabungan pengusaha rokok dan petani tembakau m...,"Rabu, 11 Sep 2024 17:31 WIB",Ekonomi
2,IHSG Melemah Tipis ke 7.760 Sore Ini,Indeks Harga Saham Gabungan (IHSG) ditutup di ...,"Rabu, 11 Sep 2024 16:37 WIB",Ekonomi
3,Rupiah Menguat Rp15.402 per Dolar AS Usai Deba...,Nilai tukar rupiah berada di level Rp15.402 pe...,"Rabu, 11 Sep 2024 16:24 WIB",Ekonomi
4,Sri Mulyani Usai Nonton Timnas-Australia: Teri...,Menteri Keuangan Sri MulyaniÂ berkomentar soal...,"Rabu, 11 Sep 2024 15:47 WIB",Ekonomi
...,...,...,...,...
95,Hasil Liga 1: PSM vs Persib Sama Kuat,PSM Makassar harus puas berbagi satu angka usa...,"Rabu, 11 Sep 2024 17:25 WIB",Olahraga
96,"Jokowi Beri Bonus Rp36,25 Miliar ke Peraih Med...",Presiden Joko Widodo (Jokowi) menyerahkan bonu...,"Rabu, 11 Sep 2024 17:13 WIB",Olahraga
97,Megawati Ungkap Target di Liga Korea: Jadi Pem...,Megawati Hangestri Pertiwi mengungkapkan targe...,"Rabu, 11 Sep 2024 16:49 WIB",Olahraga
98,Media Vietnam: Indonesia Buat Kejutan Besar La...,Media Vietnam memuji performa Timnas Indonesia...,"Rabu, 11 Sep 2024 16:24 WIB",Olahraga


In [ ]:
def crawl_article(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Memastikan permintaan berhasil
        soup = BeautifulSoup(response.content, 'html.parser')

        # Mengambil judul
        title_element = soup.find('h1', class_='text-cnn_black')
        title = title_element.get_text().strip() if title_element else 'Judul tidak ditemukan'

        # Mengambil Isi
        content_div = soup.find('div', class_='detail-text')
        content = "\n".join([p.get_text().strip() for p in content_div.find_all('p')]) if content_div else 'Isi artikel tidak ditemukan'

        # Mengambil tanggal
        date_div = soup.find('div', class_='text-cnn_grey text-sm mb-4')
        date_text = date_div.text.strip() if date_div else 'Tanggal tidak ditemukan'

        # Mengambil kategori
        category_meta = soup.find("meta", attrs={'name': 'dtk:namakanal'})
        category = category_meta['content'].strip() if category_meta and 'content' in category_meta.attrs else 'Kategori tidak ditemukan'

        return {'Judul': title, 'Isi': content, 'Tanggal': date_text, 'Kategori': category}
    except requests.RequestException as e:
        print(f"Error fetching article: {e}")
        return None

article_url = input("Masukkan URL artikel: ")

# Melakukan crawl pada satu artikel
article = crawl_article(article_url)

df = pd.DataFrame([article])  # Membuat DataFrame dari dictionary
df

Masukkan URL artikel: https://www.cnnindonesia.com/olahraga/20241015213329-142-1155726/rapor-apik-timnas-indonesia-hancur-di-tangan-china


,Judul,Isi,Tanggal,Kategori
0,Rapor Apik Timnas Indonesia Hancur di Tangan C...,Rekor apik Timnas Indonesia pada babak ketiga ...,"Selasa, 15 Okt 2024 21:40 WIB",olahraga


In [ ]:
def clean_lower(text):
    #  Memeriksa tipe data string
    if isinstance(text, str):
        return text.lower()
    return text

df['lower case'] = df['Isi'].apply(clean_lower)

# Membuat DataFrame Baru
casefolding = pd.DataFrame(df['lower case'])
casefolding

,lower case
0,rekor apik timnas indonesia pada babak ketiga ...


In [ ]:
def clean_punct(text):
    #  Memeriksa tipe data string
    if isinstance(text, str):
        # Mencocokkan angka, tanda baca, dan karakter khusus
        clean_patterns = re.compile(r'[0-9]|[/(){}\[\]\|@,;_]|[^a-z ]')
        # Mengganti karakter yang cocok dengan spasi
        text = clean_patterns.sub(' ', text)
        # Menghapus spasi berlebih dan memangkas spasi di awal dan akhir
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return text

df['tanda baca'] = df['lower case'].apply(clean_punct)

df['tanda baca']

,tanda baca
0,rekor apik timnas indonesia pada babak ketiga ...


In [ ]:
def _normalize_whitespace(text):
    # Memeriksa apakah input adalah string
    if isinstance(text, str):
        # Mengganti beberapa spasi berturut-turut dengan satu spasi
        corrected = re.sub(r'\s+', ' ', text)
        return corrected.strip()
    return text

df['spasi'] = df['tanda baca'].apply(_normalize_whitespace)

df['spasi']

,spasi
0,rekor apik timnas indonesia pada babak ketiga ...


In [ ]:
def clean_stopwords(text):
    # Memeriksa apakah input adalah string
    if isinstance(text, str):
        # Mengambil daftar stopwords dalam bahasa Indonesia
        stopword = set(stopwords.words('indonesian'))
        # Menghapus stopwords dari teks
        text = ' '.join(word for word in text.split() if word not in stopword)
        return text.strip()
    return text

df['stopwords'] = df['spasi'].apply(clean_stopwords)

df['stopwords']

,stopwords
0,rekor apik timnas indonesia babak ketiga kuali...


In [ ]:
def sastrawistemmer(text):
    factory = StemmerFactory()
    st = factory.create_stemmer()
    # Melakukan stemming pada setiap kata dalam teks
    text = ' '.join(st.stem(word) for word in tqdm(text.split()) if word in text)
    return text

df['stemming'] = df['stopwords'].apply(sastrawistemmer)

df['stemming']

100%|██████████| 141/141 [00:06<00:00, 20.33it/s]


,stemming
0,rekor apik timnas indonesia babak tiga kualifi...


In [ ]:
count_vectorizer = CountVectorizer()

if 'stemming' in df.columns:
    # Mengambil nilai dari kolom 'stemming', menghapus NaN, dan mengubahnya menjadi list
    corpus = df['stemming'].dropna().tolist()
else:
    raise KeyError("Kolom 'stemming' tidak ada di DataFrame")

# Menghitung frekuensi kata dan mengubahnya menjadi matriks
x_count = count_vectorizer.fit_transform(corpus)

# Mengambil nama fitur (kata-kata unik) dari CountVectorizer
feature_names = count_vectorizer.get_feature_names_out()

# Mengonversi matriks sparse ke dalam DataFrame pandas
x_count_df = pd.DataFrame(x_count.toarray(), columns=feature_names)

x_count_df

,abduweli,advertisement,ahmed,akibat,al,alih,apik,arab,australia,babak,...,timnas,to,tuan,ubah,unggul,video,wasit,with,yuning,zhang
0,2,1,1,2,1,2,3,1,1,1,...,5,1,1,1,1,1,1,1,2,1


In [ ]:
# Load the saved model from file
filename = 'tfidf_vectorizer.sav'
tfidf_vectorizer = pickle.load(open(filename, 'rb'))

In [ ]:
# Ambil teks dari kolom 'stemming' dan ubah ke list
corpus = df['stemming'].tolist()

# Ubah teks menjadi matriks TF-IDF
x_tfidf = tfidf_vectorizer.transform(corpus)

# Mendapatkan daftar kata unik (fitur)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Mengonversi matriks sparse ke DataFrame dan beri nama kolom sesuai dengan kata unik
tfidf_df = pd.DataFrame(x_tfidf.toarray(), columns=feature_names)

# Ambil kolom "Kategori" dari df
cat_df = df["Kategori"]

# Menambahkan kolom "Kategori" ke tfidf_df langsung dengan assign
tfidf_df['Kategori'] = cat_df.values

# Mengatur ulang kolom agar "Kategori" berada di awal
tfidf_df = tfidf_df[['Kategori'] + [col for col in tfidf_df.columns if col != 'Kategori']]

# Menampilkan DataFrame
tfidf_df

,Kategori,abroad,absolut,acara,achmad,acu,adab,adam,adaptif,adb,...,yoppy,yuan,yudha,yuran,yusuf,zayana,zona,zonasi,zulhas,zulkifli
0,olahraga,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# menggunakan label_encoder untuk merubah kata menjadi angka
label_encoder = preprocessing.LabelEncoder()
tfidf_df['Kategori'] = label_encoder.fit_transform(tfidf_df['Kategori'])

tfidf_df

,Kategori,abroad,absolut,acara,achmad,acu,adab,adam,adaptif,adb,...,yoppy,yuan,yudha,yuran,yusuf,zayana,zona,zonasi,zulhas,zulkifli
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Load the saved model from file
filename = 'lr_model.sav'
lr_model = pickle.load(open(filename, 'rb'))

In [ ]:
y_test = tfidf_df['Kategori']
x_test = tfidf_df.drop(['Kategori'], axis=1)
y_pred = lr_model.predict(x_test)

print(y_pred)

[1]


In [ ]:
# Membuat DataFrame untuk melihat nilai actual dan predicted
results_df = pd.DataFrame({'Nilai Aktual': y_test, 'Nilai Prediksi': y_pred})

results_df


,Nilai Aktual,Nilai Prediksi
0,0,1
